In [1]:
import pandas as pd
import dask
import dask.dataframe as dd
from elasticsearch import Elasticsearch
import spacy
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.tokenize import word_tokenize

c:\Users\Mohammad Dareer\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
c:\Users\Mohammad Dareer\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
c:\Users\Mohammad Dareer\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
import re
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to C:\Users\Mohammad
[nltk_data]     Dareer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Mohammad
[nltk_data]     Dareer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Mohammad
[nltk_data]     Dareer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mohammad Dareer\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Mohammad
[nltk_data]     Dareer\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
doc_df = dd.read_table('msmarco-docs.tsv\msmarco-docs.tsv', blocksize=100e6,header=None)
doc_df.columns=['docid','url','title','body']
qry_df = pd.read_table('msmarco-doctrain-queries.tsv\queries.doctrain.tsv')
qry_df.columns = ['qid', 'query']
rel_df = pd.read_table('msmarco-doctrain-top100\msmarco-doctrain-top100', delimiter=' ')
rel_df.columns = ['qid', 'Q0','docid', 'rank', 'score', 'runstring']
doc_df = doc_df.dropna()

In [4]:
qry_df=qry_df.sample(n=500,random_state=42).reset_index(drop=True)
print('Shape=>',qry_df.shape)
qry_df.head()

Shape=> (500, 2)


,qid,query
0,379306,how to set military time on iphone
1,1157310,what is a upc description
2,591004,what causes pressure skin bruising
3,635715,what does definition mean mean
4,39422,average number of bowel movements per day for ...


In [5]:
rel_df=rel_df[rel_df['qid'].isin(qry_df['qid'].unique())].reset_index(drop=True)
rel_df = rel_df.dropna()
print('Shape=>',rel_df.shape)


rel_df.head()

Shape=> (50000, 6)


,qid,Q0,docid,rank,score,runstring
0,553949,Q0,D2742163,1,-5.66638,IndriQueryLikelihood
1,553949,Q0,D3532997,2,-5.68047,IndriQueryLikelihood
2,553949,Q0,D203538,3,-5.82586,IndriQueryLikelihood
3,553949,Q0,D2768624,4,-5.84630,IndriQueryLikelihood
4,553949,Q0,D498686,5,-5.85213,IndriQueryLikelihood


In [6]:
def create_corpus(result, df):
  unique_docid=result['docid'].unique()
  condition=df['docid'].isin(unique_docid)
  corpus=df[condition].reset_index(drop=True)
  corpus=corpus.drop(columns='url')
  print('Number of Rows=>',len(corpus))
  return corpus

In [7]:
doc_corpus=create_corpus(rel_df, doc_df)
doc_corpus.head(100)

Number of Rows=> 47666


,docid,title,body
0,D2337005,Peripheral Vascular Surgery-Chapter 32,165 terms alexandriamartinez19Peripheral Vascu...
1,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...
2,D2120243,What is the largest object in our solar system?,Answers.com ® Wiki Answers ® Categories Scienc...
3,D2916861,2017 National Scout Jamboree,"by Daniel Hanson on Tuesday, March 17, 2015Jul..."
4,D2828764,Holly C.Size: 105Graded potentialsmay be eithe...,Holly C. Size: 105Graded potentialsmay be eith...
...,...,...,...
95,D810557,Chakras and Glands,"Chakras and Glands Commonly, each of the body’..."
96,D3327526,Wolfgang Mozart Biography,Quick Facts Name Wolfgang Mozart Occupation Co...
97,D2166719,Anatomy of a Perfect Facebook Post: Exactly Wh...,Research Anatomy of a Perfect Facebook Post: E...
98,D3277854,Photosynthesis and Cellular Respiration,54 terms lyv72Photosynthesis and Cellular Resp...


In [8]:
from elasticsearch import helpers
client = Elasticsearch(
    "http://localhost:9200",
     basic_auth=("elastic", "elastic")
)
resp = client.info()

In [9]:
doc_setting = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer_keyword": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": [
                        "asciifolding",
                        "lowercase",
                        "synonym",
                        "stemmer"
                    ]
                },
                "my_analyzer_shingle": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "asciifolding",
                        "lowercase",
                        "synonym",
                        "stemmer"
                    ]
                }
            },
        }
    }, "mappings": {
   
            "properties": {
                'id':{
                    'type': "string"
                },
                "doc": {
                    "type": "text",
                    "index_analyzer": "my_analyzer_keyword",
                    "search_analyzer": "my_analyzer_shingle"
                }
            }
        
    }
}

In [ ]:
setting = {
    
        "analysis": {
            "analyzer": {
                "my_analyzer_keyword": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": [
                        "asciifolding",
                        "lowercase",
                        "synonym",
                        "stemmer"
                    ]
                },
                "my_analyzer_shingle": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "asciifolding",
                        "lowercase",
                        "synonym",
                        "stemmer"
                    ]
                }
            },
        }
    }
mappings= {
        "mappings": {
   
            "properties": {
                'id':{
                    'type': "string"
                },
                "doc": {
                    "type": "text",
                    "index_analyzer": "my_analyzer_keyword",
                    "search_analyzer": "my_analyzer_shingle"
                }
            }
        
    }
}

In [ ]:
client.indices.ucreate(index= 'coretrec',mappings=mappings)

In [10]:
validate=client.index(index='trectest500', body=doc_setting)


<ipython-input-10-297435352726>:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  validate=client.index(index='trectest500', body=doc_setting)


In [20]:
i = 0
for _,rec in doc_corpus.iterrows():
    i = i + 1
    doc = {'id': rec['docid'], 'doc': rec['body']}
    client.index(index="trectest500",id = i , document=doc,)

In [178]:
print(validate)

{'_index': 'test', '_id': 'jT1aVIEB8kzoPhSCeC7o', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


In [12]:
from rake_nltk import Rake
r = Rake()

In [13]:
temp_id = 0
temp_doc_id = []
rel = {}
for i, rec in rel_df.iterrows():
    if(rec['qid'] == temp_id):
        temp_doc_id.append(rec['docid'])
    else:
        rel[temp_id] = temp_doc_id
        temp_id = rec['qid']
        temp_doc_id = []
        temp_doc_id.append(rec['docid'])


In [14]:
def precision(true:list, predect:list):
    f = 0
    for t in true :
        for p in predect:
            if p == t:
                f = f + 1
    return (f / len(predect))
def recall(true:list, predect:list):
    f = 0
    for t in true :
        for p in predect:
            if p == t:
                f = f + 1
    return f / len(true)

In [15]:
nlp = spacy.load("en_core_web_sm")
class QueryProcessor:

    def __init__(self, nlp, keep=None):
        self.nlp = nlp
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}

    def generate_query(self, text):
        doc = self.nlp(text)
        query = ' '.join(token.text for token in doc if token.pos_ in self.keep)
        return query

In [16]:
class noprocess:
    def __init__(self, keep=None):
        self.truth =[]
        self.predect =[]
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}
        self.rel = rel
        self.qry_df = qry_df

    def removeWh(self ,q):
        query = ''
        tokens =  word_tokenize(q)
        for word, tag in nltk.pos_tag(tokens):
            print(word,tag)
            if tag in  self.keep:
                query = ' '.join(word)
        return query


    def getTruth(self, id):
        doc_list = []
        for i, rec in rel_df.iterrows():
            if id == rec['qid']:
                doc_list.append(rec['docid'])
        return doc_list
    
    def exist (self, q):
        for k in self.rel.keys():
            if q == k:
                return True
        return False

    def keywordExtraction(self,q):
        r.extract_keywords_from_text(q)
        keywordList           = []
        rankedList            = r.get_ranked_phrases_with_scores()
        keywords = ''
        for keyword in rankedList:
            keyword_updated       = keyword[1].split()
            keyword_updated_string    = " ".join(keyword_updated[:2])
            keywordList.append(keyword_updated_string)
            keywords = keywords + ' ' + keyword_updated_string
            # if(len(keywordList)>9):
            #     break
        return keywords

    def searchWithExtraction(self,count, index):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        for i, rec in qry_df.iterrows():
            if not self.exist(rec['qid']):
                continue
            q = self.keywordExtraction(rec['query'])
            print(rec['query'] + ' => ' + q)
            query = {
                        "match": {
                            "doc":q 
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index=index, query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            
            truth.append(self.rel[rec['qid']])
            pi = precision(self.rel[rec['qid']], ids)
            p = p + pi
            ri = recall(self.rel[rec['qid']], ids)
            r = r + ri
            print(pi,ri)
            f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect) , f1/len(predect)

        
    def searchNoWhword(self,count, index):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        
        nlp = spacy.load("en_core_web_sm")
        for i, rec in qry_df.iterrows():
            if not self.exist(rec['qid']):
                continue
            q = QueryProcessor(nlp).generate_query(rec['query'])
            print(rec['query'] + ' => ' + q)
            query = {
                        "match": {
                            "doc":q 
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index=index, query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            
            truth.append(self.rel[rec['qid']])
            pi = precision(self.rel[rec['qid']], ids)
            p = p + pi
            ri = recall(self.rel[rec['qid']], ids)
            r = r + ri
            print(pi,ri)
            f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect) , f1/len(predect)


    
    def searchNoSW(self,count):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        for i, rec in qry_df.iterrows():
            q = remove_stopwords(rec['query'])
            if not self.exist(rec['qid']):
                continue
            query = {
                        "match": {
                            "doc":q
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index="test", query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            
            truth.append(self.rel[rec['qid']])
            pi = precision(self.rel[rec['qid']], ids)
            p = p + pi
            ri = recall(self.rel[rec['qid']], ids)
            r = r + ri
            print(pi,ri)
            #f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return  r/len(predect) , f1/len(predect)


    def searchPR(self,count):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        for i, rec in qry_df.iterrows():
            if not self.exist(rec['qid']):
                continue
            query = {
                        "match": {
                            "doc": rec['query'] 
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index="test", query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            
            truth.append(self.rel[rec['qid']])
            pi = precision(self.rel[rec['qid']], ids)
            p = p + pi
            ri = recall(self.rel[rec['qid']], ids)
            r = r + ri
            #print(pi,ri)
            f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect) , f1/len(predect)

nop = noprocess()

In [18]:
qry_df.loc[1]

qid                        1157310
query    what is a upc description
Name: 1, dtype: object

In [277]:
nop.searchPR(count=100)

(0.9019191919191917, 0.9019191919191917, 0.9019191919191917)

In [273]:
nop.searchNoWhword(count=100)

what is hq trivia game => trivia game
did obama improve the economy => obama improve economy
brake service cost estimate => brake service cost estimate
does dna replicate to prepare for cell division => replicate prepare cell division
what is involved in the socratic method? => involved socratic method
average cost to live in portland oregon => average cost live portland oregon
is farsightedness dominant or recessive => farsightedness dominant recessive
what was the first grand theft auto? => first grand theft auto
how many people died of the plague => many people died plague
causes for blood in urine => causes blood urine
where does the cork oak tree grow => cork oak tree grow
where are coatimundis from? => coatimundis
what percent of contracts do sports agents get => percent contracts do sports agents get
does cloth wiring mean knob & tube => does cloth wiring mean knob tube
when did lead free law take effect => lead free law take effect
what is a demonstrative syllogism => demonstra

(0.9042424242424237, 0.8912121212121207, 0.8951454042813957)

In [32]:
nop.searchWithExtraction(count=100, index='test2')

how to set military time on iphone =>  set military iphone
0.64 0.64
what is a upc description =>  upc description
0.97 0.97
what causes pressure skin bruising =>  causes pressure
0.2 0.2
what does definition mean mean =>  definition mean
0.49 0.49
average number of bowel movements per day for adults =>  bowel movements average number adults
0.8 0.8
is a personality disorder a mental illness =>  personality disorder mental illness
0.98 0.98
brake service cost estimate =>  brake service
0.91 0.91
average cost to redo kitchen =>  redo kitchen average cost
0.85 0.85
what is fbs stand for =>  fbs stand
0.92 0.92
meaning of anemia =>  meaning anemia
0.92 0.92
which car rental company did hertz buy =>  car rental hertz buy
0.98 0.98
identitarian lambda symbol meaning =>  identitarian lambda
0.79 0.79
what cuban sandwiches contain =>  cuban sandwiches
0.95 0.95
how quickly can a birth control test show pregnancy =>  birth control quickly
0.59 0.59
what is an avocado =>  avocado
0.99 0.99
what

(0.8820202020202024, 0.8780808080808086, 0.8797842593935102)

In [38]:
nop.searchNoWhword(count=100, index= 'test2')

how to set military time on iphone => set military time iphone
0.64 0.64
what is a upc description => upc description
0.97 0.97
what causes pressure skin bruising => causes pressure skin bruising
0.91 0.91
what does definition mean mean => definition mean mean
0.61 0.61
average number of bowel movements per day for adults => average number bowel movements day adults
0.88 0.88
is a personality disorder a mental illness => personality disorder mental illness
0.98 0.98
brake service cost estimate => brake service cost estimate
0.97 0.97
average cost to redo kitchen => average cost redo kitchen
0.85 0.85
what is fbs stand for => fbs stand
0.92 0.92
meaning of anemia => meaning anemia
0.92 0.92
which car rental company did hertz buy => car rental company did hertz buy
0.98 0.98
identitarian lambda symbol meaning => identitarian lambda symbol meaning
0.93 0.93
what cuban sandwiches contain => cuban sandwiches contain
0.88 0.88
how quickly can a birth control test show pregnancy => birth cont

(0.918787878787879, 0.914848484848485, 0.9165519361611867)

In [21]:
nop.searchNoWhword(count=100, index= 'trectest500')

how to set military time on iphone => set military time iphone
0.6 0.6
what is a upc description => upc description
0.95 0.95
what causes pressure skin bruising => causes pressure skin bruising
0.82 0.82
what does definition mean mean => definition mean mean
0.37 0.37
average number of bowel movements per day for adults => average number bowel movements day adults
0.83 0.83
is a personality disorder a mental illness => personality disorder mental illness
0.91 0.91
brake service cost estimate => brake service cost estimate
0.91 0.91
average cost to redo kitchen => average cost redo kitchen
0.79 0.79
what is fbs stand for => fbs stand
0.87 0.87
meaning of anemia => meaning anemia
0.76 0.76
which car rental company did hertz buy => car rental company did hertz buy
0.97 0.97
identitarian lambda symbol meaning => identitarian lambda symbol meaning
0.89 0.89
what cuban sandwiches contain => cuban sandwiches contain
0.73 0.73
how quickly can a birth control test show pregnancy => birth contro

(0.8359852156318677, 0.8289979959919841, 0.8301778372069588)

In [22]:
nop.searchWithExtraction(count=100, index='trectest500')

how to set military time on iphone =>  set military iphone
0.6 0.6
what is a upc description =>  upc description
0.95 0.95
what causes pressure skin bruising =>  causes pressure
0.16 0.16
what does definition mean mean =>  definition mean
0.26 0.26
average number of bowel movements per day for adults =>  bowel movements average number adults
0.77 0.77
is a personality disorder a mental illness =>  personality disorder mental illness
0.91 0.91
brake service cost estimate =>  brake service
0.78 0.78
average cost to redo kitchen =>  redo kitchen average cost
0.79 0.79
what is fbs stand for =>  fbs stand
0.87 0.87
meaning of anemia =>  meaning anemia
0.76 0.76
which car rental company did hertz buy =>  car rental hertz buy
0.96 0.96
identitarian lambda symbol meaning =>  identitarian lambda
0.66 0.66
what cuban sandwiches contain =>  cuban sandwiches
0.86 0.86
how quickly can a birth control test show pregnancy =>  birth control quickly
0.41 0.41
what is an avocado =>  avocado
0.97 0.97
wh

(0.7878489430867779, 0.7808617234468943, 0.7820415646618689)

In [17]:
class QueryExpansion:
    def __init__(self):
         self.pos_tag_map = {
            'NN': [ wn.NOUN ],
            'JJ': [ wn.ADJ, wn.ADJ_SAT ],
            'RB': [ wn.ADV ],
            'VB': [ wn.VERB ]
        }

    def tokenizer(self, sentence):
        return word_tokenize(sentence)
        
    def pos_tagger(self, tokens):
        return nltk.pos_tag(tokens)


    def stopword_treatment(self, tokens):
        stopword = stopwords.words('english')
        result = []
        for token in tokens:
            if token[0].lower() not in stopword:
                result.append(tuple([token[0].lower(), token[1]]))
        return result

    
    def pos_tag_converter(self, nltk_pos_tag):
        root_tag = nltk_pos_tag[0:2]
        try:
            self.pos_tag_map[root_tag]
            return self.pos_tag_map[root_tag]
        except KeyError:
            return ''


    def get_synsets(self,tokens):
        synsets = []
        for token in tokens:
            wn_pos_tag = self.pos_tag_converter(token[1])
            if wn_pos_tag == '':
                continue
            else:
                synsets.append(wn.synsets(token[0], wn_pos_tag))
        return synsets

    def get_tokens_from_synsets(self,synsets):
        tokens = {}
        for synset in synsets:
            for s in synset:
                if s.name() in tokens:
                    tokens[s.name().split('.')[0]] += 1
                else:
                    tokens[s.name().split('.')[0]] = 1
        return tokens


    def get_hypernyms(self,synsets):
        hypernyms = []
        for synset in synsets:
            for s in synset:
                hypernyms.append(s.hypernyms())
                
        return hypernyms

    def get_tokens_from_hypernyms(self,synsets):
        tokens = {}
        for synset in synsets:
            for s in synsets:
                for ss in s:
                    if ss.name().split('.')[0] in tokens:
                        tokens[(ss.name().split('.')[0])] += 1
                    else:
                        tokens[(ss.name().split('.')[0])] = 1
        return tokens


    def get_hyponyms(self,synsets):
        hyponyms = []
        for synset in synsets:
            for s in synset:
                hyponyms.append(s.hyponyms())
                
        return hyponyms

    def get_tokens_from_hyponyms(self,synsets):
        tokens = {}
        for synset in synsets:
            for s in synsets:
                for ss in s:
                    if ss.name().split('.')[0] in tokens:
                        tokens[(ss.name().split('.')[0])] += 1
                    else:
                        tokens[(ss.name().split('.')[0])] = 1
        return tokens

    def underscore_replacer(self,tokens):
        new_tokens = {}
        for key in tokens.keys():
            mod_key = re.sub(r'_', ' ', key)
            new_tokens[mod_key] = tokens[key]
        return new_tokens


    def generate_tokens(self,sentence):
        tokens = self.tokenizer(sentence)
        tokens = self.pos_tagger(tokens)
        tokens = self.stopword_treatment(tokens)

        synsets = self.get_synsets(tokens)
        synonyms = self.get_tokens_from_synsets(synsets)
        synonyms = self.underscore_replacer(synonyms)

        hypernyms = self.get_hypernyms(synsets)
        hypernyms = self.get_tokens_from_hypernyms(hypernyms)
        hypernyms = self.underscore_replacer(hypernyms)

        hyponyms = self.get_hyponyms(synsets)
        hyponyms = self.get_tokens_from_hyponyms(hyponyms)
        hyponyms = self.underscore_replacer(hyponyms)

        res = {**tokens, **synonyms, **hypernyms, **hyponyms}
        return res
qe =QueryExpansion()

In [49]:
print(qry_df.loc[1]['query'])
qe.generate_tokens(qry_df.loc[1]['query'])


what is a upc description


{'description': 1,
 'statement': 3,
 'speech act': 3,
 'kind': 3,
 'job description': 3,
 'specification': 3,
 'word picture': 3,
 'characterization': 3,
 'label': 3,
 'particularization': 3,
 'sketch': 3}